In [1]:
%%time
import malaya

CPU times: user 11.4 s, sys: 1.89 s, total: 13.3 s
Wall time: 18.4 s


## Text augmentation

Let say you have a very limited labelled corpus, and you want to add more, but labelling is very costly.

So, text augmentation! You can use word2vec to replace words with similar semantics!

In [2]:
string = 'saya suka makan ayam dan ikan'

In [3]:
embedded_wiki = malaya.word2vec.load_wiki()
word_vector_wiki = malaya.word2vec.word2vec(embedded_wiki['nce_weights'], 
                                            embedded_wiki['dictionary'])

In [4]:
augmented = malaya.generator.w2v_augmentation(string, 
                                  word_vector_wiki,
                                  soft=True,
                                  augment_counts=3)
augmented

['saya suka makan ayam ataupun daging',
 'saya suka makan ayam serta ikan',
 'saya suka makan ayam serta udang']

Let we compare word mover distance with original.

In [5]:
malaya.word_mover.distance(string.split(), augmented[0].split(), word_vector_wiki)

1.1325703463561534

In [6]:
malaya.word_mover.distance(string.split(), augmented[1].split(), word_vector_wiki)

0.5428173272147179

In [7]:
malaya.word_mover.distance(string.split(), augmented[2].split(), word_vector_wiki)

1.0979998614006043

They are pretty good in term of sentence similarity! **Distance that higher than 2 ratios are assumed bad**.

In [8]:
augmented = malaya.generator.w2v_augmentation('kerajaan sebenarnya sangat sayangkan rakyatnya', 
                                  word_vector_wiki,
                                  soft=True,
                                  augment_counts=5)
augmented

['kerajaan sebenarnya amat sayangkan rakyatnya',
 'kerajaan sebenarnya agak sayangkan warganya',
 'kerajaan sebenarnya semakin sayangkan rakyatnya',
 'kerajaan sebenarnya sangat sayangkan penduduknya',
 'kerajaan sebenarnya agak sayangkan penduduknya']

In [9]:
bahdanau_entities = malaya.entity.deep_model('bahdanau')
bahdanau_pos = malaya.pos.deep_model('bahdanau')

In [10]:
string = 'KUALA LUMPUR: Sempena sambutan Aidilfitri minggu depan, Perdana Menteri Tun Dr Mahathir Mohamad dan Menteri Pengangkutan Anthony Loke Siew Fook menitipkan pesanan khas kepada orang ramai yang mahu pulang ke kampung halaman masing-masing. Dalam video pendek terbitan Jabatan Keselamatan Jalan Raya (JKJR) itu, Dr Mahathir menasihati mereka supaya berhenti berehat dan tidur sebentar sekiranya mengantuk ketika memandu.'

In [11]:
result_entities = bahdanau_entities.predict(string)
result_pos = bahdanau_pos.predict(string)

## Generate ngram sentences

In [12]:
malaya.generator.sentence_ngram(string, ngram = (3, 5))

['Jalan Raya (JKJR) itu,',
 'Pengangkutan Anthony Loke Siew',
 'mengantuk ketika memandu.',
 'KUALA LUMPUR: Sempena sambutan Aidilfitri',
 'masing-masing. Dalam video pendek terbitan',
 'terbitan Jabatan Keselamatan Jalan Raya',
 'Anthony Loke Siew',
 'Jalan Raya (JKJR)',
 'Mohamad dan Menteri Pengangkutan',
 'ramai yang mahu pulang ke',
 'KUALA LUMPUR: Sempena',
 'tidur sebentar sekiranya mengantuk',
 'pesanan khas kepada',
 'Mahathir menasihati mereka supaya',
 'Raya (JKJR) itu, Dr',
 'KUALA LUMPUR: Sempena sambutan',
 'Sempena sambutan Aidilfitri minggu depan,',
 'LUMPUR: Sempena sambutan Aidilfitri minggu',
 'Loke Siew Fook menitipkan pesanan',
 'orang ramai yang mahu pulang',
 'pulang ke kampung',
 'berehat dan tidur',
 'mereka supaya berhenti berehat dan',
 'Mohamad dan Menteri',
 'Raya (JKJR) itu,',
 'Fook menitipkan pesanan khas',
 'kampung halaman masing-masing. Dalam video',
 'masing-masing. Dalam video',
 'pesanan khas kepada orang ramai',
 'halaman masing-masing. Dalam vide

## Generate ngram sentences for selected POS and Entities

In [13]:
generated_grams = malaya.generator.pos_entities_ngram(
    result_pos,
    result_entities,
    ngram = (1, 3),
    accept_pos = ['NOUN', 'PROPN', 'VERB'],
    accept_entities = ['law', 'location', 'organization', 'person', 'time'],
)
generated_grams

['Kuala Lumpur Sempena',
 'masing-masing video terbitan',
 'orang',
 'Mahathir Mohamad Menteri',
 'terbitan',
 'tidur',
 'Keselamatan Jalan',
 'Anthony Loke Siew',
 'minggu depan Perdana',
 'halaman masing-masing video',
 'sekiranya mengantuk',
 'Mohamad Menteri',
 'Tun',
 'menitipkan pesanan orang',
 'kampung halaman masing-masing',
 'masing-masing video',
 'Lumpur',
 'Kuala Lumpur',
 'orang pulang',
 'menitipkan',
 'minggu',
 'Jabatan Keselamatan Jalan',
 'berhenti',
 'Fook menitipkan',
 'Loke',
 'Menteri Tun',
 'Raya Jkjr',
 'Keselamatan',
 'Aidilfitri minggu',
 'Mohamad Menteri Pengangkutan',
 'Sempena sambutan Aidilfitri',
 'kampung halaman',
 'Raya Jkjr Dr',
 'Menteri Pengangkutan',
 'Anthony',
 'sambutan',
 'Mohamad',
 'Jalan',
 'halaman',
 'sekiranya',
 'Pengangkutan Anthony',
 'Pengangkutan',
 'Jkjr',
 'pulang',
 'berhenti berehat tidur',
 'berehat',
 'pulang kampung halaman',
 'Loke Siew Fook',
 'Mahathir',
 'Jabatan Keselamatan',
 'Jabatan',
 'berehat tidur',
 'video',
 'Jkj